In [1]:
"""
nft_tweets_demo2 (ec2 code)
"""
import requests
import pandas as pd
import numpy as np
import sqlalchemy as db
import time
import os


engine = db.create_engine("")
connection = engine.connect()
bearer_token = ''

nft_collection_df = pd.read_sql_query('select distinct(collection) from nft_tweets_demo2', connection)
nft_dict = dict(zip(nft_collection_df.collection, nft_collection_df.index))
top_100_collections = list(nft_dict.keys())
nested_top_100_collections = [top_100_collections[n:n+4] for n in range(0, len(top_100_collections), 4)]

def get_twitter_id_info(twitter_id: str):
    response = requests.get(f'https://api.twitter.com/2/users?ids={twitter_id}&user.fields=public_metrics,username', headers=headers)
    data = response.json()
    try:
        if bool(data['errors'][0]):
            return(pd.DataFrame())
    except:
        pass
    data = pd.DataFrame(data['data'])
    data = pd.concat([data.drop(['public_metrics'], axis=1), data['public_metrics'].apply(pd.Series)], axis=1)

    return data

test_count = 0

for i in range(2,len(nested_top_100_collections)): # paginating to account for twitter api rate limit
    time.sleep(900)
    for item in nested_top_100_collections[i]:
        headers = {'Authorization': f"Bearer {bearer_token}",}
        params = {
            'query': item + ' "nft" -is:retweet',
            "tweet.fields": "created_at,public_metrics,text",
            #"sort_order": "relevancy",
            "max_results": 100,
            "expansions": "author_id",
        }
        response = requests.get('https://api.twitter.com/2/tweets/search/recent', params=params, headers=headers)
        data = response.json()
        try:
            data = pd.DataFrame(data['data'])
        except:
            pass
        data = pd.concat([data.drop(['public_metrics'], axis=1), data['public_metrics'].apply(pd.Series)], axis=1)
        result = data[['created_at', 'author_id', 'text','retweet_count', 'reply_count', 'like_count', 'id']]
        result = result.replace('\n',' ', regex=True)
        result = result.replace("'"," ", regex=True)
        result = result.replace("'s", " s", regex=True)
        
        s = ', '.join(["("+ ', '.join( ["'" + item + "'", "'" + str(result.iloc[i].created_at)[0:19]+ "'",
                "'"+str(result.iloc[i].author_id)+"'", "'"+str(result.iloc[i].text)+"'", "'" + str(result.iloc[i].retweet_count)+ "'", "'"+str(result.iloc[i].reply_count)+"'", "'"+str(result.iloc[i].like_count)+"'", "'"+str(result.iloc[i].id)+"'"+ ")" ]) for i in range(len(result))])

        
        query1 = f"""
        INSERT INTO nft_tweets_demo2 
        VALUES {s}
        ON CONFLICT ON CONSTRAINT unique_tweet_id2 
        DO 
            UPDATE SET retweet_count = EXCLUDED.retweet_count
            UPDATE SET reply_count = EXCLUDED.retweet_count
            UPDATE SET like_count = EXCLUDED.like_count;
        """
        try:
            pd.read_sql_query(db.text(query1), connection)

        except:
            pass

        id_list = result['author_id'].to_list()
        id_list_final = list(set(id_list))

        for i in range(len(id_list_final)):
            df2 = get_twitter_id_info(id_list_final[i])

            if not df2.empty:
                query2 = f"""
                update nft_tweets_demo2 set username = '{df2.username.iloc[0]}', followers_count = '{df2.followers_count.iloc[0]}' where author_id = '{df2.id.iloc[0]}'
                """
            try:
                pd.read_sql_query(query2, connection)
            except:
                pass
        else:
            pass


time.sleep(18000) #every 5hr

ModuleNotFoundError: No module named 'pandas'

In [34]:
"""
trying out adding user infos too for nft_tweets_demo2
"""
import requests
import pandas as pd
import numpy as np
import sqlalchemy as db
import time
import os

engine = db.create_engine("")
connection = engine.connect()
bearer_token = ''

nft_collection_df = pd.read_sql_query('select distinct(collection) from nft_tweets_demo2', connection)
nft_dict = dict(zip(nft_collection_df.collection, nft_collection_df.index))

def get_twitter_id_info(twitter_id: str):
    response = requests.get(f'https://api.twitter.com/2/users?ids={twitter_id}&user.fields=public_metrics,username', headers=headers)
    data = response.json()
    try:
        if bool(data['errors'][0]):
            return(pd.DataFrame())
    except:
        print(data)
    data = pd.DataFrame(data['data'])
    data = pd.concat([data.drop(['public_metrics'], axis=1), data['public_metrics'].apply(pd.Series)], axis=1)

    return data

test_count = 0

for i in range(2,len(nested_top_100_collections)): # paginating to account for twitter api rate limit
    print(i)
    print(nested_top_100_collections[i])
    time.sleep(900)
    for item in nested_top_100_collections[i]:
        print("--------------" + item + "----------------")
        headers = {'Authorization': f"Bearer {bearer_token}",}
        params = {
            'query': item + ' "nft" -is:retweet',
            "tweet.fields": "created_at,public_metrics,text",
            #"sort_order": "relevancy",
            "max_results": 100,
            "expansions": "author_id",
        }
        response = requests.get('https://api.twitter.com/2/tweets/search/recent', params=params, headers=headers)
        data = response.json()
        try:
            data = pd.DataFrame(data['data'])
        except:
            print(data)
        data = pd.concat([data.drop(['public_metrics'], axis=1), data['public_metrics'].apply(pd.Series)], axis=1)
        result = data[['created_at', 'author_id', 'text','retweet_count', 'reply_count', 'like_count', 'id']]
        result = result.replace('\n',' ', regex=True)
        result = result.replace("'"," ", regex=True)
        result = result.replace("'s", " s", regex=True)
        
        s = ', '.join(["("+ ', '.join( ["'" + item + "'", "'" + str(result.iloc[i].created_at)[0:19]+ "'",
                "'"+str(result.iloc[i].author_id)+"'", "'"+str(result.iloc[i].text)+"'", "'" + str(result.iloc[i].retweet_count)+ "'", "'"+str(result.iloc[i].reply_count)+"'", "'"+str(result.iloc[i].like_count)+"'", "'"+str(result.iloc[i].id)+"'"+ ")" ]) for i in range(len(result))])

        
        query1 = f"""
        INSERT INTO nft_tweets_demo2 
        VALUES {s}
        ON CONFLICT ON CONSTRAINT unique_tweet_id2 
        DO 
            UPDATE SET retweet_count = EXCLUDED.retweet_count
            UPDATE SET reply_count = EXCLUDED.retweet_count
            UPDATE SET like_count = EXCLUDED.like_count;
        """
        try:
            pd.read_sql_query(db.text(query1), connection)

        except:
            pass

        id_list = result['author_id'].to_list()
        id_list_final = list(set(id_list))

        for i in range(len(id_list_final)):
            df2 = get_twitter_id_info(id_list_final[i])

            if not df2.empty:
                print(str(i) + " --- trial " + id_list_final[i] + " successfully added")
                query2 = f"""
                update nft_tweets_demo2 set username = '{df2.username.iloc[0]}', followers_count = '{df2.followers_count.iloc[0]}' where author_id = '{df2.id.iloc[0]}'
                """
            try:
                pd.read_sql_query(query2, connection)
            except:
                pass
        else:
            print(id_list_final[i] + " was not added")
            pass


2
['PudgyPenguins', 'MultibeastsByHaas', '8PES', 'goblintown']
--------------PudgyPenguins----------------
{'data': [{'public_metrics': {'followers_count': 731, 'following_count': 540, 'tweet_count': 3383, 'listed_count': 1}, 'name': 'DREAM2REALITY', 'id': '1514461782420590598', 'username': 'JESSERUSHING3'}]}
0 --- trial 1514461782420590598 successfully added
{'data': [{'id': '1288644227342176261', 'public_metrics': {'followers_count': 464, 'following_count': 1833, 'tweet_count': 4949, 'listed_count': 9}, 'username': 'BcSmithHere', 'name': 'BcSmithHere.eth🔴'}]}
1 --- trial 1288644227342176261 successfully added
{'data': [{'id': '1323193998', 'username': 'FreshMenuPlan', 'name': 'FreshChef 💀', 'public_metrics': {'followers_count': 619, 'following_count': 936, 'tweet_count': 2435, 'listed_count': 12}}]}
2 --- trial 1323193998 successfully added
{'data': [{'public_metrics': {'followers_count': 1154, 'following_count': 2138, 'tweet_count': 1771, 'listed_count': 3}, 'id': '91302023739779891